In [4]:
from sklearn.datasets import load_breast_cancer 
the_data = load_breast_cancer()    

In [12]:
print(the_data['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [13]:
# 0 – "benign" 
# 1 – "malignant" 
relabeled_target = 1 - the_data["target"] 

from sklearn.model_selection import train_test_split 
X = the_data["data"] 
y = relabeled_target 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

About 37% of data are attributed to class `malignant`:

In [14]:
y_train.mean(), y_test.mean()

(0.3732394366197183, 0.3706293706293706)

### Dummy classifier

In [15]:
from sklearn.dummy import DummyClassifier 
dc_mf = DummyClassifier(strategy="most_frequent") 
dc_mf.fit(X_train, y_train) 

DummyClassifier(strategy='most_frequent')

### Confusion matrix

In [58]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score
import pandas as pd

def confusion_matrix2df(y_true, y_pred):
    dc_mf_tn, dc_mf_fp, dc_mf_fn, dc_mf_tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel() 
    return pd.DataFrame([[dc_mf_tp, dc_mf_fn], [dc_mf_fp, dc_mf_tn]], columns=['Pred +', 'Pred -'], index=["True +", "True -"])

In [49]:
y_pred = dc_mf.predict(X_test)
confusion_matrix2df(y_test, y_pred)

,Pred +,Pred -
True +,0,53
True -,0,90


In [60]:
def print_metrics(y_true, y_pred):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("f1_score:", f1_score(y_true, y_pred))
    print("roc_auc_score:", roc_auc_score(y_true, y_pred))

In [61]:
print_metrics(y_true, y_pred)

Accuracy: 0.6293706293706294
Precision: 0.0
Recall: 0.0
f1_score: 0.0
roc_auc_score: 0.5


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Random Forest

In [63]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier()       
rfc.fit(X_train, y_train)       
y_true = y_test       
y_pred = rfc.predict(X_test)       
confusion_matrix2df(y_true, y_pred)

,Pred +,Pred -
True +,51,2
True -,2,88


In [65]:
print_metrics(y_true, y_pred)

Accuracy: 0.972027972027972
Precision: 0.9622641509433962
Recall: 0.9622641509433962
f1_score: 0.9622641509433962
roc_auc_score: 0.9700209643605869


### SVM

In [67]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler 
ss = StandardScaler() 
ss.fit(X_train) 
scaled_linsvc = LinearSVC(C=0.01,random_state=42) 
scaled_linsvc.fit(ss.transform(X_train), y_train) 
y_true = y_test 
y_pred = scaled_linsvc.predict(ss.transform(X_test)) 
confusion_matrix2df(y_true, y_pred)

,Pred +,Pred -
True +,50,3
True -,1,89


In [68]:
print_metrics(y_true, y_pred)

Accuracy: 0.972027972027972
Precision: 0.9803921568627451
Recall: 0.9433962264150944
f1_score: 0.9615384615384616
roc_auc_score: 0.9661425576519916


### Logistic Regression

In [69]:
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression(max_iter=10000)       
lrc.fit(X_train, y_train)       
y_true = y_test       
y_pred = lrc.predict(X_test)       
confusion_matrix2df(y_true, y_pred)

,Pred +,Pred -
True +,52,1
True -,6,84


In [70]:
print_metrics(y_true, y_pred)

Accuracy: 0.951048951048951
Precision: 0.896551724137931
Recall: 0.9811320754716981
f1_score: 0.9369369369369369
roc_auc_score: 0.9572327044025157


### Using threshold for predictions

In [78]:
import numpy as np

for threshold in np.arange(0.05, 1, step=0.05):
    probs = lrc.predict_proba(X_test)
    y_pred = (probs[:, 1] > threshold).astype(float)
    print(threshold)
    print_metrics(y_true, y_pred)

0.05
Accuracy: 0.8601398601398601
Precision: 0.726027397260274
Recall: 1.0
f1_score: 0.8412698412698413
roc_auc_score: 0.8888888888888888
0.1
Accuracy: 0.8811188811188811
Precision: 0.7571428571428571
Recall: 1.0
f1_score: 0.8617886178861789
roc_auc_score: 0.9055555555555556
0.15000000000000002
Accuracy: 0.8951048951048951
Precision: 0.7794117647058824
Recall: 1.0
f1_score: 0.8760330578512397
roc_auc_score: 0.9166666666666667
0.2
Accuracy: 0.9090909090909091
Precision: 0.803030303030303
Recall: 1.0
f1_score: 0.8907563025210083
roc_auc_score: 0.9277777777777778
0.25
Accuracy: 0.9230769230769231
Precision: 0.8387096774193549
Recall: 0.9811320754716981
f1_score: 0.9043478260869565
roc_auc_score: 0.9350104821802934
0.3
Accuracy: 0.9300699300699301
Precision: 0.8524590163934426
Recall: 0.9811320754716981
f1_score: 0.912280701754386
roc_auc_score: 0.940566037735849
0.35000000000000003
Accuracy: 0.9300699300699301
Precision: 0.8524590163934426
Recall: 0.9811320754716981
f1_score: 0.9122807017